# Simulation of an analog signal from a senor and subsequent analysis

michael@databooth.com.au

## Initial setup

Load the required packages (libraries)

In [2]:
#DONE: !pip install RPi.GPIO - only works on Raspberry Pi

In [3]:
import time
import random
from datetime import datetime, timedelta
import pandas as pd

In [4]:
try:
  import RPi.GPIO as GPIO
  GPIO.setmode(GPIO.BCM)
  GPIO.setup(18, GPIO.IN)
except Exception as e:
  print(f"Import error: RPi.GPIO - {e}")

Import error: RPi.GPIO - No module named 'RPi'


In [5]:
N_OBSERVATION = 30
MAX_TIME_EXPERIMENT_SECONDS = 1 * 60

Define any custom (user-defined) functions

In [6]:
def read_analog_sensor():
  data = random.uniform(0, 1023)  # this is returning a number in this (arbitrary range) - to simulate a reading from the sensor
  return data

In [7]:
def acquire_data(dt, data, sensor_data):
  sensor_data.append((dt, data))  # storing the data (for later use)
  return None

## Data Collection

Simulate the collection (capture) of data from the sensor for `N_SAMPLE` observations (including random times between observations)

In [8]:
def collect_data(n_observation=None, max_time=None):
  sensor_data = []
  if n_observation is None:
    n_observation = N_OBSERVATION
  if max_time is None:
    max_time = MAX_TIME_EXPERIMENT_SECONDS
  acquisition_datetime = datetime.now()
  stop_datetime = acquisition_datetime + timedelta(seconds=max_time)
  counter=0
  while counter < N_OBSERVATION and acquisition_datetime < stop_datetime:
    counter+=1
    sensor_value = read_analog_sensor()
    time_ran = random.uniform(0, 5)  # time between 0 and 5 seconds
    acquisition_datetime += timedelta(seconds=time_ran)
    print(f"{counter}: {acquisition_datetime}; Time since last (sec): {time_ran:.3f} s; Sensor value: {sensor_value:.7f}")
    acquire_data(acquisition_datetime, sensor_value, sensor_data)
    time.sleep(time_ran)
  return sensor_data

In [9]:
sensor_data = collect_data()

1: 2023-06-17 13:51:16.515922; Time since last (sec): 2.173 s; Sensor value: 194.8002377
2: 2023-06-17 13:51:17.383868; Time since last (sec): 0.868 s; Sensor value: 832.9432965
3: 2023-06-17 13:51:21.580431; Time since last (sec): 4.197 s; Sensor value: 497.5265310
4: 2023-06-17 13:51:25.883798; Time since last (sec): 4.303 s; Sensor value: 710.7957157
5: 2023-06-17 13:51:26.802853; Time since last (sec): 0.919 s; Sensor value: 367.0091035
6: 2023-06-17 13:51:28.024473; Time since last (sec): 1.222 s; Sensor value: 665.4389398
7: 2023-06-17 13:51:32.461314; Time since last (sec): 4.437 s; Sensor value: 910.9056312
8: 2023-06-17 13:51:34.549970; Time since last (sec): 2.089 s; Sensor value: 442.8602023
9: 2023-06-17 13:51:37.268177; Time since last (sec): 2.718 s; Sensor value: 1006.0899982
10: 2023-06-17 13:51:40.959720; Time since last (sec): 3.692 s; Sensor value: 189.3198455
11: 2023-06-17 13:51:42.126438; Time since last (sec): 1.167 s; Sensor value: 193.0993367
12: 2023-06-17 13:

## Transform and Analyse data

In [10]:
sensor_data

[(datetime.datetime(2023, 6, 17, 13, 51, 16, 515922), 194.8002376689207),
 (datetime.datetime(2023, 6, 17, 13, 51, 17, 383868), 832.9432965374483),
 (datetime.datetime(2023, 6, 17, 13, 51, 21, 580431), 497.5265310328642),
 (datetime.datetime(2023, 6, 17, 13, 51, 25, 883798), 710.7957157362112),
 (datetime.datetime(2023, 6, 17, 13, 51, 26, 802853), 367.0091035310198),
 (datetime.datetime(2023, 6, 17, 13, 51, 28, 24473), 665.4389398478572),
 (datetime.datetime(2023, 6, 17, 13, 51, 32, 461314), 910.9056311561924),
 (datetime.datetime(2023, 6, 17, 13, 51, 34, 549970), 442.86020234680547),
 (datetime.datetime(2023, 6, 17, 13, 51, 37, 268177), 1006.0899981521435),
 (datetime.datetime(2023, 6, 17, 13, 51, 40, 959720), 189.31984546105872),
 (datetime.datetime(2023, 6, 17, 13, 51, 42, 126438), 193.09933672379023),
 (datetime.datetime(2023, 6, 17, 13, 51, 42, 672932), 912.4392112949334),
 (datetime.datetime(2023, 6, 17, 13, 51, 44, 488212), 852.1364557005394),
 (datetime.datetime(2023, 6, 17, 13

In [ ]:
#TODO: Add some data reasonableness check here

In [11]:
def average_latency(sensor_data):
  data = pd.DataFrame(sensor_data, columns=["Collection time", "Sensor value"])
  data['time_diff'] = data["Collection time"].diff() / pd.Timedelta(seconds=1)
  return data # data["time_diff"].mean()

In [12]:
latency = average_latency(sensor_data)

In [13]:
latency

,Collection time,Sensor value,time_diff
0,2023-06-17 13:51:16.515922,194.800238,NaN
1,2023-06-17 13:51:17.383868,832.943297,0.867946
2,2023-06-17 13:51:21.580431,497.526531,4.196563
3,2023-06-17 13:51:25.883798,710.795716,4.303367
4,2023-06-17 13:51:26.802853,367.009104,0.919055
5,2023-06-17 13:51:28.024473,665.438940,1.221620
6,2023-06-17 13:51:32.461314,910.905631,4.436841
7,2023-06-17 13:51:34.549970,442.860202,2.088656
8,2023-06-17 13:51:37.268177,1006.089998,2.718207
9,2023-06-17 13:51:40.959720,189.319845,3.691543
